In [3]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

In [4]:
articles = pd.read_json('data/articles.json.xz')
google_qa = pd.read_json('data/google_qa.json.xz')
jokes = pd.read_json('data/jokes.json.xz')

In [5]:
jokes_and_not = {'text': [], 'funny': []}

counter = 0
for line in articles.text[:5]:
    jokes_and_not['text'].append(line)
    jokes_and_not['funny'].append(0)

for line in google_qa.text[:5]:
    jokes_and_not['text'].append(line)
    jokes_and_not['funny'].append(0)

for line in jokes.text[:5]:
    jokes_and_not['text'].append(line)
    jokes_and_not['funny'].append(1)

In [8]:
df = pd.DataFrame(jokes_and_not)

In [10]:
df.head(15)


,text,funny
0,The New York Times reported on Thursday that T...,0
1,"Entitled “Passions”, the autobiography documen...",0
2,A so-called tiered deposit rate would mean ban...,0
3,Air Force Brigade General Miguel Sisco Mora wa...,0
4,Much of Hudson’s Bay’s value is locked up in i...,0
5,which is the most common use of opt-in e-mail ...,0
6,how i.met your mother who is the mother Tracy ...,0
7,what type of fertilisation takes place in huma...,0
8,who had the most wins in the nfl Active quarte...,0
9,what happened to the lost settlement of roanok...,0
